# I. $LU$ factorization of a square matrix

Consider a simple naive implementation of the LU decomposition. 

Note that we're using the `numpy` arrays to represent matrices [do **not** use `np.matrix`].

In [1]:
import numpy as np

def diy_lu(a):
    """Construct the LU decomposition of the input matrix.
    
    Naive LU decomposition: work column by column, accumulate elementary triangular matrices.
    No pivoting.
    """
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u

In [2]:
# Now, generate a full rank matrix and test the naive implementation

import numpy as np

N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

np.linalg.matrix_rank(a)


6

In [3]:
# Tweak the printing of floating-point numbers, for clarity
np.set_printoptions(precision=3)

In [4]:
L, u = diy_lu(a)

print(L, "\n")
print(u, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print(L@u - a)

[[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    1.455 1.    0.    0.    0.   ]
 [1.    1.714 1.742 1.    0.    0.   ]
 [1.    1.882 2.276 2.039 1.    0.   ]
 [1.    2.    2.671 2.944 2.354 1.   ]] 

[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -1.125e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  0.000e+00  2.625e-01  4.574e-01  5.975e-01  7.013e-01]
 [ 0.000e+00  1.110e-16  0.000e+00 -2.197e-02 -4.480e-02 -6.469e-02]
 [ 0.000e+00 -2.819e-16  0.000e+00  0.000e+00  8.080e-04  1.902e-03]
 [ 0.000e+00  3.369e-16  0.000e+00 -1.541e-18  2.168e-19 -1.585e-05]] 

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  2.220e-16 -1.110e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00  2.220e-16 -5.551e-17 -1.665e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00 -1.110e-16  2.776e-16 -2.776e-16  5.551e-17]
 

# II. The need for pivoting

Let's tweak the matrix a little bit, we only change a single element:

In [5]:
a1 = a.copy()
a1[1, 1] = 3

Resulting matix still has full rank, but the naive LU routine breaks down.

In [6]:
np.linalg.matrix_rank(a1)

6

In [7]:
l, u = diy_lu(a1)

print(l, u)

[[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]] [[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]


C:\ProgramData\Miniconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel import kernelapp as app
C:\ProgramData\Miniconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in matmul
C:\ProgramData\Miniconda3\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in matmul
C:\ProgramData\Miniconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


### Test II.1

For a naive LU decomposition to work, all leading minors of a matrix should be non-zero. Check if this requirement is satisfied for the two matrices `a` and `a1`.

(20% of the grade)

In [11]:
def test_minor(a):
    assert a.shape[0] == a.shape[1], 'non-quadratic'
    n = a.shape[0]
    i = 1
    while i < n:
        b = a[:i][:i]
        if np.linalg.matrix_rank(b) != i:
            return i + 1
        i+=1
    return 0

    

In [12]:
#test

n = 3
a = np.zeros((n,n),dtype=float)
for i in range(n): 
    for j in range(n):
        a[i][j] = 1
        
k=1
a1 = np.zeros((n,n),dtype=float)
for i in range(n):
    for j in range(n):
        a1[i][j] = k
        k+=1
        
t = test_minor(a)
t1 = test_minor(a1)

if t == 0:
    print('all leading minors of matrix a are non-zero')
else: 
    print('leading minor #{0} of matrix a is zero'.format(t))
    

if t1 == 0:
    print('all leading minors of matrix a1 are non-zero')
else: 
    print('leading minor #{0} of matrix a1 is zero'.format(t1))
    

leading minor #3 of matrix a is zero
all leading minors of matrix a1 are non-zero


### Test II.2

Modify the `diy_lu` routine to implement column pivoting. Keep track of pivots, you can either construct a permutation matrix, or a swap array (your choice).

(40% of the grade)

Implement a function to reconstruct the original matrix from a decompositon. Test your routines on the matrices `a` and `a1`.

(40% of the grade)

In [20]:
def lu_pivot(a):
    N = a.shape[0]
    u = a.copy()
    L = np.eye(N)
    swap_array = []
    for j in range(N-1):
        piv = u[j, :]
        maxi = 0
        lam = np.eye(N)
        i = j
        while i < (N):
            if abs(u[i][j]) > abs(piv[j]):
                piv = u[i, :]
                maxi = i
            i+=1
        if maxi > j:
            u[[j, maxi], :] = u[[maxi, j],:]
            swap_array.append([j, maxi])
        else:
            swap_array.append([1,1])
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u, swap_array




In [21]:
L, u, swap = lu_pivot(a1)
print(L)
print(u)

[[1.    0.    0.   ]
 [0.571 1.    0.   ]
 [0.143 2.    1.   ]]
[[7.    8.    9.   ]
 [0.    0.429 0.857]
 [0.    0.    0.   ]]


In [22]:
def restore(L, u):
    return L @ u

In [30]:
L1, u1, swap_arr = lu_pivot(a1)
L1 @ u1

[[0, 2], [1, 1]]

In [24]:
L1, u1, swap_arr = lu_pivot(a1)
b1 = restore(L1, u1)
print(a1 - b1)


[[-6. -6. -6.]
 [ 0.  0.  0.]
 [ 6.  6.  6.]]
